In [1]:
import tensorflow as tf

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
import sys
sys.path.insert(1,'/wsu/home/gy/gy40/gy4065/hm.jetscapeml.source')
sys.path.insert(1,'/content/drive/My Drive/Projects/110_JetscapeMl/hm.jetscapeml.source')
sys.path.insert(1,'/content/drive/MyDrive/Projects/110_JetscapeMl/hm.jetscapeml.source')
sys.path.insert(1,'/g/My Drive/Projects/110_JetscapeMl/hm.jetscapeml.source')
sys.path.insert(1,'G:\\My Drive\\Projects\\110_JetscapeMl\\hm.jetscapeml.source')

In [3]:
# loading libraries
print('Loading/Installing Package => Begin\n\n')
import jet_ml_dataset_builder.jet_ml_dataset_builder_utilities as util
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import set_directory_paths
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import parse_parameters
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import load_dataset
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import install
# install("trimesh")
import os
from time import time
import glob
# import trimesh
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, History

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import json
tf.random.set_seed(1234)
print('\nLoading/Installing Package => End\n\n')

Loading/Installing Package => Begin



Loading/Installing Package => End




In [4]:
print('\n########################################################################')
print('Checking the running platforms\n')
# Call the function and retrieve the dataset_directory_path and simulation_directory_path
# Access the dataset_directory_path and simulation_directory_path
dataset_directory_path, simulation_directory_path = set_directory_paths()
print('########################################################################\n')


########################################################################
Checking the running platforms

Python version: 3.11.5
OS: Windows
OS version: 10
running on Colab: False
Dataset Directory Path: D:\Projects\110_JetscapeMl\hm.jetscapeml.data\
Simulation Results Path: D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\
########################################################################



In [5]:
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import get_label_items
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import get_labels_str
label_items_dict = get_label_items()
label_str_dict=get_labels_str(label_items_dict)

Aggregatring all parameters values
label_items:
 {'y_class_label_items': ['MMAT', 'MLBT'], 'alpha_s_items': [0.2, 0.3, 0.4], 'q0_items': [1.5, 2.0, 2.5]}
Building required params for the loading the dataset file
labels_str:
 {'class_labels_str': 'MMAT_MLBT', 'alpha_s_items_str': '0.2_0.3_0.4', 'q0_items_str': '1.5_2.0_2.5'}


In [6]:
from jet_ml_dataset_builder.jet_ml_dataset_builder_utilities import get_dataset
dataset_x, dataset_y=get_dataset(1000,label_str_dict, dataset_directory_path,working_column=1)

Loading the whole dataset
dataset.x: <class 'numpy.ndarray'> 1024000 (1000, 32, 32)
dataset.y: <class 'numpy.ndarray'> 3000 (1000, 3)
Extract the working column#1 for classification
dataset.x: <class 'numpy.ndarray'> 1024000 (1000, 32, 32)
dataset.y: <class 'numpy.ndarray'> 1000 (1000,)
dataset.y(working_column) sample ['0.4' '0.4' '0.2' '0.4' '0.4' '0.3' '0.4' '0.3' '0.4' '0.4']


In [ ]:
from jet_ml_models.pointnet import build_pointnet_classifier_model
from jet_ml_models.pointnet import compile_pointnet_classifier_model_with_hyperparam
NUM_POINTS = 1024
#because alpha_s can get 3 values
NUM_CLASSES = 3
activation="softmax"
# activation="sigmoid"


pointnet=build_pointnet_classifier_model(NUM_POINTS=NUM_POINTS,NUM_CLASSES=NUM_CLASSES, activation=activation)


In [7]:
learning_rate=0.001
pointnet=compile_pointnet_classifier_model_with_hyperparam(pointnet,learning_rate)

Model: "pointnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1024, 3)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1024, 32)  │        128 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 1024, 32)  │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1024, 32)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1024, 64)  │      2,112 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1024, 64)  │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 1024, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1024, 512) │     33,280 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1024, 512) │      2,048 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 1024, 512) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ activation_2[0][… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    131,328 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 9)         │      1,161 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3, 3)      │          0 │ dense_2[0][0]   

 Total params: 748,076 (2.85 MB)

 Trainable params: 741,996 (2.83 MB)

 Non-trainable params: 6,080 (23.75 KB)

Optimizer:  <keras.src.optimizers.adam.Adam object at 0x0000025C99893350>
Loss function:  sparse_categorical_crossentropy
Metrics:  ['loss', 'compile_metrics']


In [8]:
from jet_ml_models.pointnet import preprocess_dataset
(x_train,  y_train,x_test,  y_test)=preprocess_dataset(dataset_x, dataset_y)

Pre-processing
dataset_x_points shape: (1000, 1024, 3)
deleting the original dataset after splitting ...
train_points: <class 'numpy.ndarray'> 2457600 (800, 1024, 3)
train_y: <class 'numpy.ndarray'> 800 (800,)
x_test_points: <class 'numpy.ndarray'> 614400 (200, 1024, 3)
y_test: <class 'numpy.ndarray'> 200 (200,)
Preprocess y_train and y_test
Encoding to sparse categorical variable
y_test_encoded:
 [2 0 1 1 2 2 1 0 1 0]


In [9]:
def augment(points, label):
    # jitter points
    points += keras.random.uniform(points.shape, -0.005, 0.005, dtype="float64")
    # shuffle points
    points = keras.random.shuffle(points)
    return points, label

In [10]:
BATCH_SIZE = 32
from tensorflow import data as tf_data
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf_data.Dataset.from_tensor_slices((x_test, y_test))
train_dataset_size = int(len(dataset) * train_size)

dataset = dataset.shuffle(len(x_train)).map(augment)
test_dataset = test_dataset.shuffle(len(x_test)).batch(BATCH_SIZE)

train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [11]:
print(len(train_dataset))
print(len(test_dataset))

20
7


In [12]:
monitor = 'val_accuracy'  # 'val_accuracy' or 'val_loss'
n_epochs = 20

print("simulation_directory_path:",simulation_directory_path)
simulation_path=f'{simulation_directory_path}jetml_pointnet_classification_alpha_s_{label_str_dict["alpha_s_items_str"]}'
print("simulation_path:",simulation_path)
current_simulation_name=f'_size_{1000}'
current_simulation_path=simulation_path+current_simulation_name
current_simulation_name=f'_epochs_{n_epochs}'
current_simulation_path=current_simulation_path+current_simulation_name
print(current_simulation_path)

fold=1
current_simulation_name=f'_fold_{fold}'
# fold_path=f'{current_simulation_path}_fold_{fold}'
current_simulation_path=current_simulation_path+current_simulation_name
# Use ModelCheckpoint callback to save the best model
best_model_file_path = f'{current_simulation_path}_best_model.keras'
print("best_model_file_path:",best_model_file_path)

simulation_directory_path: D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\
simulation_path: D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\jetml_pointnet_classification_alpha_s_0.2_0.3_0.4
D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\jetml_pointnet_classification_alpha_s_0.2_0.3_0.4_size_1000_epochs_20
best_model_file_path: D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\jetml_pointnet_classification_alpha_s_0.2_0.3_0.4_size_1000_epochs_20_fold_1_best_model.keras


In [13]:
# !pip install --upgrade tensorflow


In [14]:
pointnet.fit(train_dataset, epochs=20, validation_data=validation_dataset)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 52.3641 - sparse_categorical_accuracy: 0.3396 - val_loss: 35.9114 - val_sparse_categorical_accuracy: 0.3812
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 35.6972 - sparse_categorical_accuracy: 0.4007 - val_loss: 35.5482 - val_sparse_categorical_accuracy: 0.3375
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 35.6654 - sparse_categorical_accuracy: 0.3469 - val_loss: 35.7209 - val_sparse_categorical_accuracy: 0.2937
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 35.6295 - sparse_categorical_accuracy: 0.3097 - val_loss: 36.0093 - val_sparse_categorical_accuracy: 0.3375
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 35.4332 - sparse_categorical_accuracy: 0.3565 - val_loss: 35.6086 - val_sparse_categorical_accuracy: 0.3812
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 35.4195 - sparse_categorical_accuracy: 0.3721 - val_loss: 36.3312 - val_sparse_categorical_accuracy: 0.3750
Epoc

In [17]:
from jet_ml_models.pointnet import train_model_with_callbacks
# %%timeit -n1 -r1
monitor='val_sparse_categorical_accuracy'
model, history, train_time=train_model_with_callbacks(pointnet, x_train, y_train, x_test, y_test, monitor, best_model_file_path, n_epochs)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 34.9952 - sparse_categorical_accuracy: 0.3197
Epoch 1: val_sparse_categorical_accuracy improved from inf to 0.38000, saving model to D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\jetml_pointnet_classification_alpha_s_0.2_0.3_0.4_size_1000_epochs_20_fold_1_best_model.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 34.9968 - sparse_categorical_accuracy: 0.3203 - val_loss: 35.9771 - val_sparse_categorical_accuracy: 0.3800
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 34.9848 - sparse_categorical_accuracy: 0.3255
Epoch 2: val_sparse_categorical_accuracy improved from 0.38000 to 0.36000, saving model to D:\Projects\110_JetscapeMl\hm.jetscapeml.data\simulation_results\jetml_pointnet_classification_alpha_s_0.2_0.3_0.4_size_1000_epochs_20_fold_1_best_model.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 34.9864 - sparse_categorical_accuracy: 0.3262 - val_loss: 37.5524 - val_sparse_categorical_accur

In [ ]:
print(train_time)
print(model)
print(history.history)

In [ ]:
from jet_ml_models.pointnet import save_training_history
training_history_file_path_json,training_history_file_path_csv,training_history_file_path_csv = \
  save_training_history(history,current_simulation_path)

In [ ]:
from jet_ml_models.pointnet import plot_training_history
plot_training_history_path=plot_training_history(history,current_simulation_path)
print(plot_training_history_path)

In [ ]:
from jet_ml_models.pointnet import evaluate_model
accuracy, confusion_matrix = evaluate_model(model, x_test, y_test)